In [1]:
import torch
import torch.nn as nn
import numpy as np
import gym

In [2]:
# 超参数
BATCH_SIZE = 32
LR = 0.01                   # learning rate
EPSILON = 0.9               # 最优选择动作百分比
GAMMA = 0.9                 # 奖励递减参数
TARGET_REPLACE_ITER = 100   # Q 现实网络的更新频率
MEMORY_CAPACITY = 2000      # 记忆库大小
env = gym.make('CartPole-v0')   # 立杆子游戏
env = env.unwrapped
N_ACTIONS = env.action_space.n  # 杆子能做的动作
N_STATES = env.observation_space.shape[0]   # 杆子能获取的环境信息数

In [3]:
class Net(nn.Module):
    def __init__(self,):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(N_STATES, 10)
        self.fc1.weight.data.normal_(0, 0.1)
        self.out = nn.Linear(10, N_ACTIONS)
        self.out.weight.data.normal_(0, 0.1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = tensor.relu(x)
        actions_value = self.out(x)
        return actions_value

In [ ]:
class DQN(object):
    def __init__(self,):
        self.eval_net, self.target_net = Net(), Net()
        
        self.learn_step_counter = 0 # 用于 target 更新计时
        self.memory_counter = 0 # 记忆库记数
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES*2+2))
        self.optimizer = torch.optim.Adam(
            self.eval_net.parameters(), lr=LR)
        self.loss_func = nn.MSELoss() # 误差公式
        
    def choose_action(self, x):
        x = torch.unsqueeze(torch.FloatTensor(x), 0)
        if np.random.uniform() < EPSILON:
            actions_value = self.eval_net.forward(x)
            # API待查
            action = torch.max(actions_value, 1)[1].data.numpy()[0,0]
        else:
            action = np.random.randint(0, N_ACTIONS)
        return action
        
    def store_transition(self, s, a, r, s_):
        transition = np.hstack((s, [a, r], s_))
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index, :] = transition
        self.memory_counter += 1
        
    def learn(self):
        
        